In [1]:
import numpy as np
from envs import *

from tensorflow.contrib import keras
from keras.layers import Input, Embedding, Flatten, Dense, Dropout
from keras.layers import Dot, Concatenate, Activation
from keras.models import Model, Sequential

from time import sleep
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
class Env_2:
    
    def __init__(self):
        self.data = req_reset(envs='second', verbose=False)
        self.n_model = 0
    
    def fit_model_1(self):
        """
        embedding model, with inputs of "user_id", "item_id" and "variables"
        """
        self.n_model = 1
        
        user_id_input = Input(shape=[1], name='user')
        item_id_input = Input(shape=[1], name='item')
        variables_input = Input(shape=[5], name='variables')

        embedding_size = 32
        user_embedding = Embedding(output_dim=embedding_size, input_dim=2000,
                           input_length=1, name='user_embedding')(user_id_input)
        item_embedding = Embedding(output_dim=embedding_size, input_dim=2000,
                           input_length=1, name='item_embedding')(item_id_input)

        user_vecs = Flatten()(user_embedding)
        item_vecs = Flatten()(item_embedding)

        concatenated = Concatenate(axis=-1)([user_vecs, item_vecs, variables_input])
        concatenated = Dense(32, activation='relu')(concatenated)
        concatenated = Dropout(0.5)(concatenated)
        outputs = Dense(1)(concatenated)

        self.model = Model(inputs=[user_id_input, item_id_input, variables_input], outputs=outputs)
        self.model.compile(optimizer='adam', loss='mse')
        
        X = [self.data['user_history'], self.data['item_history'], self.data['variables_history']]
        y = self.data['rating_history']
        
        self.model.fit(X, y, epochs=50, verbose=False)
        return
    
    def fit_model_2(self):
        """
        sequential model, only use "variables" as the input
        """
        self.n_model = 2
        
        self.model = Sequential()
        self.model.add(Dense(5,input_dim=5, activation='relu'))
        self.model.add(Dropout(0.25))
        self.model.add(Dense(2, activation='relu'))
        self.model.add(Dropout(0.25))
        self.model.add(Dense(1))

        self.model.compile(optimizer='adam',loss='mse')
        
        X = np.array(self.data['variables_history'])
        y = self.data['rating_history']
        self.model.fit(X, y, epochs=50, verbose=False)
        return
    
    def test(self, n_iter=500, online=True):
        """
        test model's performance
        ---------------------------
        n_iter: 'int', times of applying "req_predict" function to get test data
        
        online: 'bool', "True" means after each time of prediction, update the model
                        "False" means don't update model
        """
        if self.n_model == 1:
            next_X = [[self.data['next_user']], [self.data['next_item']], [self.data['next_variables']]]
        else:
            next_X = np.array(self.data['next_variables']).reshape(1,-1)
        next_predict = self.model.predict(next_X)
        mse, mae = 0, 0

        for i in tqdm(range(n_iter)):
            sleep(0.05)
            test = req_predict(next_predict, envs='second', verbose=False)
            if self.n_model == 1:
                next_X = [[test['next_user']], [test['next_item']], [test['next_variables']]]
            else:
                next_X =  np.array(test['next_variables']).reshape(1,-1)
            next_y = [test['rating']]
            next_predict = self.model.predict(next_X)
    
            mse += (next_y - next_predict) ** 2
            mae += np.abs(next_y - next_predict)
            
                        
            if online:
                self.model.fit(next_X,next_y,verbose=False)

        print('mse: ', float(mse)/n_iter)
        print('mae: ', float(mae)/n_iter)
        return
    
    def predict(self, input_data):
        """
        perdict a rating
        """
        if self.n_model == 1:
            pred = self.model.predict([[input_data['next_user']], [input_data['next_item']], [input_data['next_variables']]])
        else:
            pred = self.model.predict(np.array(input_data['next_variables']).reshape(1,-1))
        return float(pred)

In [3]:
env = Env_2()
env.fit_model_1()
env.test()

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:34<00:00, 14.37it/s]


mse:  2.4091898225345174
mae:  1.2984950094223022


In [4]:
env = Env_2()
env.fit_model_2()
env.test()

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:37<00:00, 13.55it/s]


mse:  2.0841696582958655
mae:  1.258834035396576
